# IBM Applied Data Science Capstone Course by Coursera

## OPENING A NEW SHOPPING MALL IN DELHI, INDIA

### 1. Import Libraries

In [19]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import csv

print('Libraries imported.')

Libraries imported.


### 2. Get Neighbourhood Data from Wikipedia

In [20]:
#send the GET request - PAGE 1
data = requests.get("https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi").text
#print(data)

In [21]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'lxml')
#print(soup)

In [22]:

def createCSV():
  csv_file = open('Delhi1.csv', 'w')
  csv_writer = csv.writer(csv_file)
  csv_writer.writerow(['Neighbourhood'])
  uls = soup.find_all("ul")
  for ul in uls:
    lis = ul.find_all("li")
    for li in lis:
      a = li.find('a')
      if a: 
        ti = a.get('title')
        if ti: 
        #print(ti)
          csv_writer.writerow([ti])
  csv_file.close()

In [23]:
createCSV()
# After creating CSV there were some unnecessory data points 
# i.e unnecessory information also get included so I had to remove it manually.
# So new CSV is "Delhi.csv"


## 3. Creating the dataframe

In [42]:
delhi_df = pd.read_csv('Delhi.csv', error_bad_lines=False)
print(delhi_df.shape)
delhi_df.head(10)

(116, 1)


,Neighbourhood
0,Adarsh Nagar
1,Ashok Vihar
2,Karala
3,Model Town
4,Narela
5,Pitam Pura
6,Shalimar Bagh
7,Civil Lines
8,Gulabi Bagh
9,Kamla Nagar


## 4. Getting the Geographical coordinates

In [43]:
#!pip install geocoder
import geocoder # to get coordinates
geolocator = Nominatim()

def get_latlng(neighborhood):
  location = geolocator.geocode('{}, Delhi, India'.format(neighborhood))
  if location:
    lat_lng_coords = [location.latitude, location.longitude]
    return lat_lng_coords

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [44]:
# call the function to get the coordinates, store in a new list

coords = []

for neighborhood in delhi_df["Neighbourhood"].tolist():
  coords.append(get_latlng(neighborhood))

print(coords[:5])
print(len(coords))

Adarsh Nagar, Bindapur, Dwarka Tehsil, South West Delhi, Delhi, India
(28.6141925, 77.0715411848447)


## 5. Adding Lat and Long to the DataFrame

In [51]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

df_coords.head()

,Latitude,Longitude
0,28.614193,77.071541
1,28.699453,77.184826
2,28.735140,77.032511
3,28.702714,77.193991
4,28.842610,77.091835


In [54]:
# merge the coordinates into the original dataframe
delhi_df['Latitude'] = df_coords['Latitude']
delhi_df['Longitude'] = df_coords['Longitude']

delhi_df.head()
#delhi_df.shape

,Neighbourhood,Latitude,Longitude
0,Adarsh Nagar,28.614193,77.071541
1,Ashok Vihar,28.699453,77.184826
2,Karala,28.735140,77.032511
3,Model Town,28.702714,77.193991
4,Narela,28.842610,77.091835


In [55]:
# save the DataFrame as CSV file
delhi_df.to_csv("Delhi_final.csv", index=False)

## 6. Create a map of Delhi with Neighbourhoods


In [58]:
# get the coordinates of Delhi, India
address = 'Delhi, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi, India is ({}, {}).'.format(latitude, longitude))

The geograpical coordinate of Delhi, India is (28.6517178, 77.2219388).


In [63]:
# create map of Delhi using latitude and longitude values
map_Delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(delhi_df['Latitude'], delhi_df['Longitude'], delhi_df['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Delhi)  
    
map_Delhi

In [64]:
# save the map as HTML file
map_Delhi.save('map_Delhi.html')

## 7. Using Foursquare API to explore the neighbourhoods


In [102]:
#FOURSQUARE API DETAILS
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


## Now, Let's get the top 30 venues that are within a radius of 1000 meters

In [73]:
radius = 1000
venues = []

for lat, lng, neighborhood in zip(delhi_df['Latitude'], delhi_df['Longitude'], delhi_df['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    #print(results)
    #return only relevant information for each nearby venue
    for venue in results:
        #if venue:
        venues.append((
            neighborhood,
            lat, 
            lng, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [74]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1813, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
1,Adarsh Nagar,28.614193,77.071541,Uttam nagar,28.620201,77.068709,Metro Station
2,Adarsh Nagar,28.614193,77.071541,Gold's Gym A Block Janakpuri,28.622439,77.069348,Gym
3,Adarsh Nagar,28.614193,77.071541,Potholes at Dabri,28.605309,77.072504,Pool
4,Ashok Vihar,28.699453,77.184826,Bellagio,28.696361,77.180021,Asian Restaurant


## Let's check how many venues were returned for each neighorhood

In [76]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adarsh Nagar,4,4,4,4,4,4
Alaknanda,27,27,27,27,27,27
Anand Vihar,15,15,15,15,15,15
Ashok Nagar,30,30,30,30,30,30
Ashok Vihar,10,10,10,10,10,10
Badarpur,5,5,5,5,5,5
Bali Nagar,27,27,27,27,27,27
Barakhamba Road,27,27,27,27,27,27
Chanakyapuri,23,23,23,23,23,23


## Let's find out how many unique categories can be curated from all the returned venues

In [77]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 191 uniques categories.


In [78]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Metro Station', 'Gym', 'Pool',
       'Asian Restaurant', 'Snack Place', 'Sandwich Place', 'Donut Shop',
       'Pizza Place', 'Coffee Shop', 'South Indian Restaurant',
       'Fast Food Restaurant', 'Food Truck', 'Department Store',
       'Convenience Store', 'Ice Cream Shop', 'Bank',
       'Frozen Yogurt Shop', 'Bakery', 'Track Stadium',
       'Chinese Restaurant', 'Mobile Phone Shop', 'Park',
       'Light Rail Station', 'Bus Station', 'Restaurant', 'Hotel',
       'Smoke Shop', 'Italian Restaurant', 'Café', 'Burger Joint',
       'Flea Market', 'Lake', 'Mini Golf', 'Train Station',
       'Breakfast Spot', 'Playground', 'Miscellaneous Shop',
       'Dumpling Restaurant', 'Grocery Store', 'Parking', 'Boutique',
       'American Restaurant', 'Historic Site', 'Hostel', 'Cricket Ground',
       'History Museum', 'Road', 'Movie Theater', 'Soccer Stadium'],
      dtype=object)

In [79]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

## 8. Analyze Each Neighbourhood

In [80]:
# one hot encoding
Del_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Del_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Del_onehot.columns[-1]] + Del_onehot.columns[:-1].values.tolist()
Del_onehot = Del_onehot[fixed_columns]

print(Del_onehot.shape)
Del_onehot.head()

(1813, 192)


,Neighborhoods,ATM,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parking,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racetrack,Record Shop,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Travel Lounge,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [82]:
Del_grouped = Del_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(Del_grouped.shape)
Del_grouped.head(10)

(113, 192)


,Neighborhoods,ATM,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parking,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racetrack,Record Shop,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Travel Lounge,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adarsh Nagar,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,Alaknanda,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.074074,0.0,0.0,0.074074,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,

In [84]:
len(Del_grouped[Del_grouped["Shopping Mall"] > 0])

25

## Create a new DataFrame for Shopping Mall data only

In [85]:
Del_mall = Del_grouped[["Neighborhoods","Shopping Mall"]]
Del_mall.to_csv('Delhi_mall_data.csv')

## 9. Cluster Neighbourhoods

In [86]:
# set number of clusters
kclusters = 3

Del_clustering = Del_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Del_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 2, 0, 0, 2, 0, 2, 0], dtype=int32)

In [87]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Del_merged = Del_mall.copy()

# add clustering labels
Del_merged["Cluster Labels"] = kmeans.labels_

In [89]:
Del_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
Del_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Adarsh Nagar,0.000000,0
1,Alaknanda,0.000000,0
2,Anand Vihar,0.066667,2
3,Ashok Nagar,0.066667,2
4,Ashok Vihar,0.000000,0


In [94]:
# merge with kolkata_data to add latitude/longitude for each neighborhood
Del_merged = Del_merged.join(delhi_df.set_index("Neighbourhood"), on="Neighborhood")

print(Del_merged.shape)
Del_merged.head() # check the last columns!

(114, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adarsh Nagar,0.000000,0,28.614193,77.071541
1,Alaknanda,0.000000,0,28.529336,77.251632
2,Anand Vihar,0.066667,2,28.641115,77.312502
3,Ashok Nagar,0.066667,2,28.636021,77.101822
4,Ashok Vihar,0.000000,0,28.699453,77.184826


In [95]:
# sort the results by Cluster Labels
print(Del_merged.shape)
Del_merged.sort_values(["Cluster Labels"], inplace=True)
Del_merged

(114, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adarsh Nagar,0.000000,0,28.614193,77.071541
76,Patel Nagar,0.000000,0,28.659809,77.156957
75,Paschim Vihar,0.000000,0,28.669578,77.095956
73,Palam,0.000000,0,28.591893,77.082824
72,Paharganj,0.000000,0,28.641499,77.214061
71,Old Delhi Railway Station,0.000000,0,28.660905,77.227715
70,Okhla,0.000000,0,28.563662,77.289055
68,Nizamuddin West,0.000000,0,28.588365,77.244955
67,New Usmanpur,0.000000,0,28.683855,77.256005
66,New Friends Colony,0.000000,0,28.567101,77.269764


## Now create a map to visualize the clusters

In [96]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = ['red','blue','darkgreen']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Del_merged['Latitude'], Del_merged['Longitude'], Del_merged['Neighborhood'], Del_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [97]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## 9. Examine Clusters

## Cluster 0

In [99]:
Del_merged.loc[Del_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adarsh Nagar,0.0,0,28.614193,77.071541
76,Patel Nagar,0.0,0,28.659809,77.156957
75,Paschim Vihar,0.0,0,28.669578,77.095956
73,Palam,0.0,0,28.591893,77.082824
72,Paharganj,0.0,0,28.641499,77.214061
71,Old Delhi Railway Station,0.0,0,28.660905,77.227715
70,Okhla,0.0,0,28.563662,77.289055
68,Nizamuddin West,0.0,0,28.588365,77.244955
67,New Usmanpur,0.0,0,28.683855,77.256005
66,New Friends Colony,0.0,0,28.567101,77.269764


## Cluster 1

In [100]:
Del_merged.loc[Del_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
16,Delhi Cantonment,0.200000,1,28.593833,77.134979
55,Moti Bagh,0.250000,1,28.667619,77.184227
47,Laxmi Nagar,0.166667,1,28.630553,77.277575
74,Pandav Nagar,0.142857,1,28.650024,77.153676
88,Sarai Rohilla,0.250000,1,28.667873,77.190267


## Cluster 2

In [101]:
Del_merged.loc[Del_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
90,Sarojini Nagar,0.047619,2,28.574157,77.195370
49,Lodi Colony,0.045455,2,28.590702,77.220921
109,Vishwas Nagar,0.111111,2,28.664436,77.294960
2,Anand Vihar,0.066667,2,28.641115,77.312502
3,Ashok Nagar,0.066667,2,28.636021,77.101822
50,Malviya Nagar,0.066667,2,28.533920,77.212447
44,Kirti Nagar,0.062500,2,28.653281,77.141773
6,Bali Nagar,0.074074,2,28.654138,77.128178
56,Moti Nagar,0.066667,2,28.657859,77.142429
64,Netaji Nagar,0.037037,2,28.573534,77.186359


# **Observation**

Most of the shopping malls are concentrated in the central east, southern and few on the northern part of Delhi, with 
- The highest number in cluster 1 
- Moderate number in cluster 2. 
- On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods.

This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central eastern and southern part of the city, with the suburb area still have very few shopping malls.

**Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition.** Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 2 with moderate competition.

Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of shopping malls and suffering from intense competition.